In [1]:
import numpy as np
from helpers import *
from implementations import *
from data_processing import *
from hyperparams import *

from classification import *
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload 

In [3]:
# some functions used below

def get_only_accuracy(y_result, y_te):
    difference = (y_result-y_te)
    good_guess = difference[difference==0]
    bad_guess = difference[difference!=0]
    accuracy = len(good_guess)/(len(good_guess)+len(bad_guess))
    return accuracy

def standardize(x):
    X = np.copy(x)
    means = np.mean(X, axis=0)
    tx_new = X - means * np.ones(np.shape(X))
    std_dev = np.std(tx_new, axis=0)
    X = tx_new / (std_dev * np.ones(np.shape(X)))
    return X, means, std_dev

## Loading/splitting train data

In [4]:
# loading train data
yb, input_data, ids = load_csv_data("train.csv")
dimensions = np.shape(input_data)
N = dimensions[0]
P = dimensions[1]
yb = np.reshape(yb,[N,1])
yb[yb==-1] = 0 # adapting to our logistic loss function

In [5]:
x = data_replace(input_data)

In [6]:
y_tr, x_tr, y_te, x_te = split_data(yb,x,0.8)

## Handling case 1

It is immediately noticeable that this method is completely incorrect: the -999 values and others that are too high make the exponential terms overflow 

In [7]:
x1 = np.copy(x)
initial_w = np.ones([P,1])
max_iters = 100
gamma = 0.7
w_opt_1, mse = logistic_regression(yb, x1, initial_w, max_iters, gamma)

/Users/toye/Desktop/ML1_final/implementations.py:229: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-t))
/Users/toye/Desktop/ML1_final/implementations.py:246: RuntimeWarning: divide by zero encountered in log
  loss = -(1/len(y)) * ( y.T @ np.log(sigmoid(tx@w)) + (1-y).T @ np.log(1-sigmoid(tx@w)) )
/Users/toye/Desktop/ML1_final/implementations.py:246: RuntimeWarning: invalid value encountered in matmul
  loss = -(1/len(y)) * ( y.T @ np.log(sigmoid(tx@w)) + (1-y).T @ np.log(1-sigmoid(tx@w)) )


## Handling case 2 

In [8]:
x2 = np.copy(x_tr)

x2, means2, std_dev2 = standardize(x2)

initial_w = np.zeros([P,1])
max_iters = 100
gamma = 0.7
w_opt_2, mse = logistic_regression(y_tr, x2, initial_w, max_iters, gamma)

In [9]:
tx_new = np.copy(x_te)
mean_te_2 = np.mean(tx_new)
std_te_2 = np.std(tx_new)
tx_new = tx_new - mean_te_2
x_te_2 = tx_new / std_te_2  

temporary = sigmoid(x_te_2@w_opt_2)
y_result = temporary
y_result[y_result>0.5] = 1
y_result[y_result<0.5] = 0
accuracy2 = get_only_accuracy(y_result, y_te)

In [10]:
accuracy2

0.46056

## Case 3

In [11]:
x3 = np.copy(x_tr)
x3, means3, std_dev3 = standardize(x3)
x3 = add_w0(x3,x3.shape[0])

initial_w = np.zeros([P+1,1])
max_iters = 100
gamma = 0.7
w_opt_3, mse = logistic_regression(y_tr, x3, initial_w, max_iters, gamma)

In [12]:
tx_new = x_te - means3
x_te_3 = tx_new / std_dev3  
x_te_3 = add_w0(x_te, x_te_3.shape[0])

temporary = sigmoid(x_te_3@w_opt_3)

y_result = temporary
y_result[y_result>0.5] = 1
y_result[y_result<0.5] = 0
accuracy3 = get_only_accuracy(y_result, y_te)

In [13]:
accuracy3

0.52396

## Case 4

In [14]:
x4 = np.copy(x_tr)
indices_min_max =[3,11,12,22,26]
indices_gaussian =[0,1,6,8,13,14,16,17,24,27]
indices_angles = [15,18,20,25,28]
indices_gaussian_log = [2,5,7,9,10,19]
normalize(x4, indices_gaussian_log, indices_angles, indices_gaussian, indices_min_max)
x4= add_w0(x4,x4.shape[0])

initial_w = np.zeros([P+1,1])
max_iters = 100
gamma = 0.7
w_opt_4, mse = logistic_regression(y_tr, x4, initial_w, max_iters, gamma)

In [15]:
x_te_4 = np.copy(x_te)
normalize(x_te_4, indices_gaussian_log, indices_angles, indices_gaussian, indices_min_max)
x_te_4 = add_w0(x_te_4,x_te_4.shape[0])

temporary = sigmoid(x_te_4@w_opt_4)
y_result = temporary
y_result[y_result>0.5] = 1
y_result[y_result<0.5] = 0
accuracy4 = get_only_accuracy(y_result, y_te)

In [16]:
accuracy4

0.34424

## Case 5

In [17]:
x5 = np.copy(x_tr)
indices_min_max =[3,11,12,22,26]
indices_gaussian =[0,1,6,8,13,14,16,17,24,27]
indices_angles = [15,18,20,25,28]
indices_gaussian_log = [2,5,7,9,10,19]
normalize(x5, indices_gaussian_log, indices_angles, indices_gaussian, indices_min_max)
x5 = np.delete(x5,29,1)
x5 = np.delete(x5,23,1)
x5 = np.delete(x5,22,1)
x5 = np.delete(x5,21,1)
x5 = np.delete(x5,4,1)
x5= add_w0(x5, x5.shape[0])


initial_w = np.zeros([P-4,1])
max_iters = 100
gamma = 0.7
w_opt_5, mse = logistic_regression(y_tr, x5, initial_w, max_iters, gamma)


In [18]:
x_te_5 = np.copy(x_te)
normalize(x_te_5, indices_gaussian_log, indices_angles, indices_gaussian, indices_min_max)
x_te_5 = np.delete(x_te_5,29,1)
x_te_5 = np.delete(x_te_5,23,1)
x_te_5 = np.delete(x_te_5,22,1)
x_te_5 = np.delete(x_te_5,21,1)
x_te_5 = np.delete(x_te_5,4,1)
x_te_5= add_w0(x_te_5,x_te_5.shape[0])

temporary = sigmoid(x_te_5@w_opt_5)
y_result = temporary
y_result[y_result>0.5] = 1
y_result[y_result<0.5] = 0
accuracy5 = get_only_accuracy(y_result, y_te)

In [19]:
accuracy5

0.7431

In [20]:
def get_accuracy(y_results, y_te, score):
    """Checks whether prediction are accurate by compraing with y_te
    
    Args: 
        predictions:
        y_te:
    
    Returns:
        len(good_guess):
        len(bad_guess):
    """ 
    
    difference = (y_results-y_te)
    good_guess = difference[difference==0]
    bad_guess = difference[difference!=0]
    accuracy = len(good_guess)/(len(good_guess)+len(bad_guess))
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for i in range(y_results.shape[0]):
        if difference[i] == 1:
            FP +=1
        if difference[i] == -1:
            FN +=1
        else :
            if y_results[i] == 1:
                TP +=1
            else:
                TN +=1       
                
    precision = TP/(TP+FP)  
    recall = TP/(TP+FN)
    auc = get_auc(score, y_te)
    #print(f"How well our model can classify binary outcomes: accuracy of {accuracy}, precision of {precision}, and recall of {recall}")
    print("How well our model can classify binary outcomes: accuracy of %.3f, precision of %.3f, recall of %.3f, and AUC score of %.3f" % (accuracy, precision, recall, auc))
    
    return accuracy, precision, recall


def get_auc(score, y_results):

    y = y_results

    # false positive rate
    FPR = []
    # true positive rate
    TPR = []
    # Iterate thresholds from 0.0 to 1.0
    thresholds = np.arange(0.0, 1.01, 0.001)
    print(len(thresholds))

    # get number of positive and negative examples in the dataset
    P = sum(y)
    N = len(y) - P

    # iterate through all thresholds and determine fraction of true positives
    # and false positives found at this threshold
    for thresh in thresholds:
        FP=0
        TP=0
        thresh = round(thresh,2) 
        for i in range(len(score)):
            if (score[i] >= thresh):
                if y[i] == 1:
                    TP += 1
                if y[i] == 0:
                    FP += 1            
        FPR = np.append(FPR,FP/N)
        TPR = np.append(TPR, TP/P)

    #computing Arean Under Curve using the trapezoidal method
    auc = -1 * np.trapz(TPR, x=FPR)
    print(auc)

    
    plt.plot(FPR, TPR, marker='.', color='darkorange', label='ROC curve', clip_on=False)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label = 'No Discrimination')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.title('ROC curve, AUC = %.2f'%auc)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig('AUC_example.png')
    plt.show()
    
    return auc